In [ ]:
!pip install firebase_admin
!pip install requests
!pip install -U scikit-fuzzy # http://pythonhosted.org/scikit-fuzzy/auto_examples/index.html

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

In [ ]:
cred = credentials.Certificate('/tmp/iothiya-firebase-adminsdk-neqv5-341ced2a56.json')

In [ ]:
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://iothiya-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

%matplotlib inline

In [ ]:
# Antecedents
hum= ctrl.Antecedent(np.arange(0, 100), 'hum')
temp = ctrl.Antecedent(np.arange(0, 40), 'temp')

# Consequents
motor_level = ctrl.Consequent(np.arange(0, 12), 'motor_level')

In [ ]:
# Humidity memberships
hum['low'] = fuzz.trapmf(hum.universe, [0, 0, 40, 50])
hum['normal'] = fuzz.trapmf(hum.universe, [40, 50, 60, 70])
hum['high'] = fuzz.trapmf(hum.universe, [60, 70, 100, 100])

In [ ]:
# Temperature memberships
temp['low'] = fuzz.trapmf(temp.universe, [0, 0, 17, 18])
temp['normal'] = fuzz.trapmf(temp.universe, [17, 18, 22, 25])
temp['high'] = fuzz.trapmf(temp.universe, [22, 25, 26, 30])
temp['very-high'] = fuzz.trapmf(temp.universe, [26, 30, 40, 40])

In [ ]:
# Motor Level memberships
motor_level['off'] = fuzz.trapmf(motor_level.universe, [0, 0, 2, 3])
motor_level['weak'] = fuzz.trapmf(motor_level.universe, [2, 3, 5, 6])
motor_level['normal'] = fuzz.trapmf(motor_level.universe, [5, 6, 8, 9])
motor_level['strong'] = fuzz.trapmf(motor_level.universe, [8, 9, 11, 12])

In [ ]:
# Rule system
rule1 = ctrl.Rule((hum['low'] & temp['low']), motor_level['off'])


In [ ]:
rule2 = ctrl.Rule((hum['normal'] & temp['low']), motor_level['off'])


In [ ]:
rule3 = ctrl.Rule((hum['high'] & temp['low']), motor_level['weak'])


In [ ]:
rule4 = ctrl.Rule((hum['low'] & temp['normal']), motor_level['off'])


In [ ]:
rule5 = ctrl.Rule((hum['normal'] & temp['normal']), motor_level['weak'])


In [ ]:
rule6 = ctrl.Rule((hum['high'] & temp['normal']), motor_level['weak'])


In [ ]:
rule7 = ctrl.Rule((hum['low'] & temp['high']), motor_level['normal'])


In [ ]:
rule8 = ctrl.Rule((hum['normal'] & temp['high']), motor_level['normal'])


In [ ]:
rule9 = ctrl.Rule((hum['high'] & temp['high']), motor_level['strong'])


In [ ]:
rule10 = ctrl.Rule((hum['low'] & temp['very-high']), motor_level['normal'])


In [ ]:
rule11 = ctrl.Rule((hum['normal'] & temp['very-high']), motor_level['strong'])


In [ ]:
rule12 = ctrl.Rule((hum['high'] & temp['very-high']), motor_level['strong'])

In [ ]:
motor_level_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4,
                                       rule5, rule6, rule7, rule8,
                                       rule9, rule10, rule11, rule12])
motor_level_output = ctrl.ControlSystemSimulation(motor_level_ctrl)

In [ ]:
trigger = True
while(trigger):
  try:
    hum_value = db.reference('/data-sensor/hum').get()
    temp_value = db.reference('/data-sensor/temp').get()
    motor_level_value = db.reference('/data-actuator')

    motor_level_output.input['temp'] = float(temp_value)
    motor_level_output.input['hum'] = float(hum_value)

    motor_level_output.compute()

    print(motor_level_output.output['motor_level'])
    if (motor_level_output.output['motor_level'] < 3):
        level = 0
    elif (motor_level_output.output['motor_level'] > 2 and motor_level_output.output['motor_level'] < 6):
        level = 1
    elif motor_level_output.output['motor_level'] > 5 and motor_level_output.output['motor_level'] < 9:
        level = 2
    elif motor_level_output.output['motor_level'] > 8:
        level = 3

    try:
        motor_level_value.set({
          'level':level
        })
    except NameError:
        print(NameError)
    else:
        print("Success")
  except KeyboardInterrupt:
    level = 0
    motor_level_value.set({
          'level':level
        })
    print("Interrupt!")
    trigger = False

9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.053057313371406
Success
9.0530573133